In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
# para pasar el primet test
from werkzeug.security import generate_password_hash, check_password_hash
import threading
import re


import jwt  # Para generar tokens
import datetime
from functools import wraps  # Para proteger rutas


app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///base.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = "supersecretkey"  # Clave para firmar los tokens
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # password = db.Column(db.String(120), nullable=False)
    # Aumentamos el tamaño para bcrypt para el primer test
    password = db.Column(db.String(256), nullable=False) 
    # agragamos un rol para el test test_authorization
    role = db.Column(db.String(10), nullable=False, default="user")  # "admin" o "user"

with app.app_context():
  db.create_all()


# Función para validar y limpiar el username
def sanitize_username(username):
    if re.search(r'[<>"/\'&]', username):
        return None
    return username

def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = request.headers.get('Authorization')
        if not token:
            return jsonify({"message": "Token is missing"}), 401
        
        try:
            token = token.split(" ")[1]  # Remover "Bearer"
            data = jwt.decode(token, app.config['SECRET_KEY'], algorithms=["HS256"])
            request.user_role = data["role"]  # Guardamos el rol en la petición
            print(f"User role from token: {request.user_role}")  # Debug
        except:
            print(f"Token error: {e}")  # Debug
            return jsonify({"message": "Invalid token"}), 401
        
        return f(*args, **kwargs)
    return decorated

@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(username=data['username']).first()
    
    if not user or not check_password_hash(user.password, data['password']):
        return jsonify({"message": "Invalid credentials"}), 401

    token = jwt.encode({
        "user": user.username,
        "role": user.role, 
        "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=1)
    }, app.config['SECRET_KEY'], algorithm="HS256")

    return jsonify({"token": token})


@app.route('/')
def home():
    return "Welcome to the Security Testing Demo!"

@app.route('/users', methods=['GET'])
@token_required
def get_users():
    users = User.query.all()
    return jsonify([{"id": user.id, "username": user.username, "password": user.password , "role":user.role} for user in users])

@app.route('/user/<int:id>', methods=['GET'])
@token_required  # Aseguramos que se requiere un token para acceder a esta ruta
def get_user(id):
    # Verificamos que el ID solicitado coincida con el ID del usuario en el token
    if request.user_role != "admin" and id != request.user_role:
        return jsonify({"message": "Forbidden"}), 403
    user = User.query.get(id)
    if user:
        # return jsonify({"id": user.id, "username": user.username, "password": user.password})
        return jsonify({"id": user.id, "username": user.username,})
    return jsonify({"message": "User not found"}), 404

@app.route('/user', methods=['POST'])
def add_user():
    
    data = request.get_json()

    # Validar que los datos necesarios estén presentes
    if not data.get('username') or not data.get('password'):
        return jsonify({"message": "Username and password are required"}), 400
            
    # Validar username (no permitir caracteres sospechosos)
    if "'" in data['username'] or ";" in data['username'] or "--" in data['username']:
        return jsonify({"message": "Invalid characters in username"}), 400

    # Encriptar contraseña antes de almacenarla
    hashed_password = generate_password_hash(data['password'])
    
    # new_user = User(username=data['username'], password=data['password'])
    # pasamos la contraseña
    clean_username = sanitize_username(data['username'])
    if clean_username is None:
        return jsonify({"message": "Invalid characters in username"}), 400

    role = data.get("role", "user")  # Si no hay rol, asignamos "user"
    
    new_user = User(username=clean_username, password=hashed_password, role=role)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({"message": "User added successfully", "id":new_user.id}), 201

@app.route('/user/<int:id>', methods=['PUT'])
@token_required
def update_user(id):
    print(f"Updating user {id} - Role: {request.user_role}")  # Debug
    
    if request.user_role != "admin":
        return jsonify({"message": "Forbidden"}), 403
    
    data = request.get_json()
    user = User.query.get(id)
    if user:
        user.username = data['username']
        user.password = data['password']
        db.session.commit()
        return jsonify({"message": "User updated successfully"})
    return jsonify({"message": "User not found"}), 404

@app.route('/user/<int:id>', methods=['DELETE'])
def delete_user(id):
    user = User.query.get(id)
    if user:
        db.session.delete(user)
        db.session.commit()
        return jsonify({"message": "User deleted successfully"})
    return jsonify({"message": "User not found"}), 404


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5001}).start()